# Desktop Vector Skin Version User Preferences

Pending resolution of identified Prefupdate bugs, I reviewed the mediawiki [user_properties table](https://www.mediawiki.org/wiki/Manual:User_properties_table) to determine the total number of users with each vector skin preference set for each of the test wiki. Note: Unlike PrefUpdate, this does not record every change in users preference but stores any current non-default state of the user's preference. As result, this data may included users that enabled and disabled their vectorskin version preference multiple times. 


## Data Notes:

This reflects all current nondefault user preferences. If a user has not made any changes to their vector skin version then there will be no record for that user in this table and skin preference is default (i.e. Modern for logged-in users on test wikis)
* Only accounts for logged-in users.
* Data reflects the current state and does not account for users that opt in and opt out multiple times since deployment.
* Based on the context that the new vector skin was deployed as the default setting for all logged-in users on the wikis below, we can assume each of the values mean the following:
    * Legacy: These are users that have currently opt'd out of the modern version.
    * Modern: These are users that opt'd out and then opt'd back in to the modern version. (Note: This number does not reflect the total number of users that are using the modern skin; only the users that made changes to their default preferences)
    * Unknown: There were a few VectorSkinVersion values set to 0 (instead of 1 [legacy] or 2[modern]). I need to further investigate what those values indicate.

## Remaining To Dos:
 - Investigate what `VectorSkinVersion` set to 0 means.
 - Look into options of putting on superset dashboard.
 - Pending resolution of PrefUpdate bugs or new schema, show opt-out rate over time.
 - Confirm user table does not include only users. 

In [47]:
from wmfdata import mariadb
import pandas as pd
import numpy as np

In [139]:
query = """ 
SELECT 
    up_value as skin,
    COUNT(*) as users
FROM user_properties
WHERE 
    up_property = 'VectorSkinVersion'
GROUP BY up_value """

In [150]:
#define list of target wikis
wikis = ['euwiki','frwiktionary', 'ptwikiversity', 'fawiki','hewiki', 'frwiki']

In [141]:
up_skin=list()
for wiki in wikis:
    prefs = mariadb.run(
      query.format(),
      wiki
    )
    up_skin.append(prefs)

skin= pd.concat(up_skin)

In [142]:
skin_users = skin['users'].sum()
print('Total number of users for whom we have vector skin preferences set in the user_properties table:' , skin_users)

Total number of users for whom we have vector skin preferences set in the user_properties table: 226605


In [143]:
skin_aliases = {
    "0":"unknown",
    "1":"legacy",
    "2":"modern"
}

skin= skin.replace({"skin": skin_aliases})

# Number of users for each skin type overall

In [144]:
user_skin=skin.groupby('skin').sum()
user_skin

,users
skin,
legacy,128242
modern,98334
unknown,29


# Number of users for each skin type by wiki

In [145]:
#List of wikis to correspond to data values 
wikis_list = ['euwiki','euwiki', 'frwiktionary',  'frwiktionary',  'frwiktionary', 'ptwikiversity', 'ptwikiversity','fawiki', 'fawiki', 'fawiki','hewiki', 'hewiki', 'hewiki', 'frwiki', 'frwiki', 'frwiki']

In [146]:
skin['wiki'] = wikis_list

In [147]:
skin

,skin,users,wiki
0,legacy,2067,euwiki
1,modern,1852,euwiki
0,unknown,6,frwiktionary
1,legacy,4684,frwiktionary
2,modern,3899,frwiktionary
0,legacy,480,ptwikiversity
1,modern,462,ptwikiversity
0,unknown,2,fawiki
1,legacy,21442,fawiki
2,modern,22046,fawiki


In [148]:
user_skin_bywiki=pd.pivot_table(skin, index=['wiki','skin'],values=['users'],aggfunc=np.sum)

In [149]:
user_skin_bywiki

users
wiki          skin          
euwiki        legacy    2067
              modern    1852
fawiki        legacy   21442
              modern   22046
              unknown      2
frwiki        legacy   71535
              modern   49185
              unknown     19
frwiktionary  legacy    4684
              modern    3899
              unknown      6
hewiki        legacy   28034
              modern   20890
              unknown      2
ptwikiversity legacy     480
              modern     462

## Total Number of Users on Test Wikis

We can use the total number of users on test wikis using the [mediawiki user table](https://www.mediawiki.org/wiki/Manual:User_table) to estimate the opt-out rate.

In [155]:
# collect total number of users on each wiki

query = """ 
SELECT 
    COUNT(DISTINCT user_id) AS num_users
FROM user"""


In [156]:
user_count = mariadb.run(commands = query, dbs = wikis, format="pandas")

In [157]:
user_count['wiki'] = wikis

In [158]:
user_count

,num_users,wiki
0,115786,euwiki
1,291868,frwiktionary
2,29455,ptwikiversity
3,963101,fawiki
4,685306,hewiki
5,3905482,frwiki


## Calculating Opt Out Rate

The opt-out rate was calculated by dividing the total number of users with their vector version preference changed to 'legacy' by the total number of all registered users on the wiki. 

In [166]:
# Create list of legacy users - these are all users that opt-out assuming modern is the default

legacy_users = skin[skin['skin']=='legacy']
#rename colums

legacy_users.columns = ['skin', 'num_legacy_users', 'wiki']


In [172]:
# join to user_count table to obtain opt-out rate for each wiki

opt_out_rate = legacy_users.merge(user_count, left_on = 'wiki', right_on = 'wiki')
opt_out_rate

,skin,num_legacy_users,wiki,num_users
0,legacy,2067,euwiki,115786
1,legacy,4684,frwiktionary,291868
2,legacy,480,ptwikiversity,29455
3,legacy,21442,fawiki,963101
4,legacy,28034,hewiki,685306
5,legacy,71535,frwiki,3905482


In [174]:
# Calculate opt-out rate

opt_out_rate['pct_opt_out_rate'] = opt_out_rate['num_legacy_users']/ opt_out_rate['num_users'] * 100

opt_out_rate

,skin,num_legacy_users,wiki,num_users,opt_out_rate,pct_opt_out_rate
0,legacy,2067,euwiki,115786,0.017852,1.785190
1,legacy,4684,frwiktionary,291868,0.016048,1.604835
2,legacy,480,ptwikiversity,29455,0.016296,1.629604
3,legacy,21442,fawiki,963101,0.022264,2.226350
4,legacy,28034,hewiki,685306,0.040907,4.090727
5,legacy,71535,frwiki,3905482,0.018317,1.831656
